In [1]:
from trustifai import Trustifai, MetricContext

# Confidence for LLM response

In [ ]:
trust_engine = Trustifai(None, "config_file.yaml")

In [ ]:
response = trust_engine.generate("what was the major means of transport in the year 2150? Why it got failed?")

In [ ]:
response

{'response': 'As of my knowledge cutoff in June 2024, there is no historical or factual information about the year 2150 since it is in the future. Therefore, I cannot provide details about the major means of transport in 2150 or why it might have failed.\n\nIf you are asking for a speculative or fictional scenario, I can certainly help create one. Would you like me to imagine what transportation might be like in 2150 and reasons why a particular mode might have failed?',
 'metadata': {'confidence_score': 0.77,
  'confidence_label': 'Medium Confidence',
  'confidence_details': {'explanation': 'Model shows moderate uncertainty.',
   'avg_logprob': -0.18,
   'variance': 0.08,
   'token_count': 97},
  'logprobs_available': True}}

# Trust Score for RAG response

In [ ]:
from langchain_core.documents import Document

query = "What is Acme Corp's policy on remote work?"

answer = """Acme Corp operates on a fully remote-first policy with no office requirements. 
Employees can work from anywhere in the world and the company has closed all 
physical office locations as of January 2024. They provide a $2000 annual 
stipend for coworking spaces and have implemented a 4-day work week.
"""


documents = [
    Document(
        page_content="Acme Corp announced a hybrid work model in 2023, requiring employees to be in office 3 days per week.",
        metadata={"source": "hr_policy_2023.pdf"}
    ),
    Document(
        page_content="The company provides home office stipends of up to $500 for remote setup.",
        metadata={"source": "benefits_guide.pdf"}
    )
]
# documents = ["New Delhi is the capital of India. It houses the parliament.",
#              "The Eiffel Tower is the most famous monument in Paris, France.",
#              "India's capital city, New Delhi, is known for the Red Fort."]

#supports document objects like langchain/llamaindex documents, list, dictionary etc.

metric_context = MetricContext(
    query=query,
    answer=answer,
    documents=documents,
)
trust_engine = Trustifai(metric_context, "config_file.yaml")

In [ ]:
trust_score = trust_engine.get_trust_score()

In [4]:
trust_score

{'score': 0.46,
 'label': 'UNRELIABLE',
 'details': {'evidence_coverage': {'score': 0.0,
   'label': 'Likely Hallucinated Answer',
   'details': {'explanation': 'Many claims lack support from source documents.',
    'strategy': 'LLM',
    'total_sentences': 3,
    'supported_sentences': 0,
    'unsupported_sentences': ['Acme Corp operates on a fully remote-first policy with no office requirements.',
     'Employees can work from anywhere in the world and the company has closed all \nphysical office locations as of January 2024.',
     'They provide a $2000 annual \nstipend for coworking spaces and have implemented a 4-day work week.'],
    'failed_checks': 0}},
  'semantic_drift': {'score': 0.91,
   'label': 'Strong Alignment',
   'details': {'explanation': 'Answer semantically aligned with source documents.',
    'total_documents': 2}},
  'consistency': {'score': 0.88,
   'label': 'Stable Consistency',
   'details': {'explanation': 'Model produces highly consistent responses.',
    's

In [4]:
rg = trust_engine.build_reasoning_graph(trust_score)

In [5]:
rg

ReasoningGraph(trace_id='825b7a31-6bdd-46c8-be4c-e57f85e88985', nodes=[ReasoningNode(node_id='evidence_coverage', node_type='metric', name='Evidence Coverage', inputs={}, outputs={}, score=0.0, label='Likely Hallucinated Answer', details={'explanation': 'Many claims lack support from source documents.', 'strategy': 'LLM', 'total_sentences': 3, 'supported_sentences': 0, 'unsupported_sentences': ['Acme Corp operates on a fully remote-first policy with no office requirements.', 'Employees can work from anywhere in the world and the company has closed all \nphysical office locations as of January 2024.', 'They provide a $2000 annual \nstipend for coworking spaces and have implemented a 4-day work week.'], 'failed_checks': 0}), ReasoningNode(node_id='semantic_drift', node_type='metric', name='Semantic Drift', inputs={}, outputs={}, score=0.91, label='Strong Alignment', details={'explanation': 'Answer semantically aligned with source documents.', 'total_documents': 2}), ReasoningNode(node_id

In [6]:
trust_engine.visualize(rg)

<class 'pyvis.network.Network'> |N|=6 |E|=5

In [7]:
print(trust_engine.visualize(rg,"mermaid"))

```mermaid
flowchart TD
   evidence_coverage["<b>Evidence Coverage</b><br/>Score: 0.00<br/>Likely Hallucinated Answer"]
   semantic_drift["<b>Semantic Drift</b><br/>Score: 0.91<br/>Strong Alignment"]
   consistency["<b>Consistency</b><br/>Score: 0.87<br/>Stable Consistency"]
   source_diversity["<b>Source Diversity</b><br/>Score: 0.85<br/>High Trust"]
   trust_aggregation{"<b>Trust Score</b><br/>Score: 0.46"}
   final_decision("<b>Decision: UNRELIABLE</b>")
    evidence_coverage --> trust_aggregation
    semantic_drift --> trust_aggregation
    consistency --> trust_aggregation
    source_diversity --> trust_aggregation
    trust_aggregation --> final_decision
    style evidence_coverage fill:#ff6b6b,color:#000000
    style semantic_drift fill:#2ecc71,color:#000000
    style consistency fill:#2ecc71,color:#000000
    style source_diversity fill:#2ecc71,color:#000000
    style trust_aggregation fill:#ff6b6b,color:#000000
    style final_decision fill:#ff6b6b,color:#000000
```
